# LSTM text classifiation model
<hr>

We will build a text classification model using LSTM model on the TREC Dataset. 

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import random
from nltk.corpus import stopwords, twitter_samples
# from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import KFold
from nltk.stem import PorterStemmer
from string import punctuation
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
import time

%config IPCompleter.greedy=True
%config IPCompleter.use_jedi=False
# nltk.download('twitter_samples')

In [2]:
corpus = pd.read_csv("places_names_labelled.csv", index_col = 0)
# corpus.label = corpus.label.astype(int)
print(corpus.shape)
corpus

(25161, 8)


,name,clean_name,place_id,vicinity,no_of_ratings,avg_rating,type_1,purpose
0,Colombo Swimming Club,colombo swimming club,ChIJETFLQ0FZ4joRwioIp4MnzHA,148 “Storm Lodge” Galle Road 3,1181.0,4.4,tourist_attraction,recreational
1,Storm lodge,storm lodge,ChIJGSU_sGtZ4joRiyZjgglUlPE,"148 “Storm Lodge” Galle Road 3, Colombo 00300",NaN,NaN,lodging,personal
5,Ayura,ayura,ChIJQwy6bEFZ4joRU9m1ylxsbMs,"142, Yathama Building, Galle Road, Colombo",16.0,4.5,jewelry_store,shopping
6,Baby Gallery Kurulapina,baby gallery kurulapina,ChIJR1dqFUFZ4joRsqACvaqRNws,"WR8X+655, Colombo",NaN,NaN,clothing_store,shopping
7,"Galle Face , Taj Hotel",galle face taj hotel,ChIJo015E0FZ4joRdGUr8gWxo4E,"138 Colombo - Galle Main Road, Colombo",2.0,5.0,restaurant,dining
...,...,...,...,...,...,...,...,...
44395,Ichiro Motoring - Wijerama,ichiro motoring wijerama,ChIJk3QsSX1a4joRDO-HD30GZT8,"High Level Road, Nugegoda",60.0,3.5,car_repair,personal
44396,Softlogic Glomark Supermarket,softlogic glomark supermarket,ChIJD436Ynxb4joR_tn_grvgpNs,"Nugegoda Delkanda Apartment, High Level Road, ...",1278.0,4.2,grocery_or_supermarket,shopping
44397,Softlogic Glomark Supermarket,softlogic glomark supermarket,ChIJD436Ynxb4joR_tn_grvgpNs,"Nugegoda Delkanda Apartment, High Level Road, ...",1278.0,4.2,grocery_or_supermarket,shopping
44398,Mirai Auto Land (Pvt) Ltd,mirai auto land pvt ltd,ChIJ6V8_E2da4joREMxUiWN0uVA,"Delkanda, 383 High Level Rd, Avissawella Road,...",52.0,4.4,car_repair,personal


In [3]:
corpus['purpose'].replace(corpus['purpose'].unique(), list(range(0, len(corpus['purpose'].unique()))), inplace=True)

In [4]:
corpus = corpus[~(corpus['purpose']==6)]

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(corpus,corpus['purpose'],test_size=0.2, random_state=32)

In [6]:
# Define a function to compute the max length of sequence
def max_length(sequences):
    '''
    input:
        sequences: a 2D list of integer sequences
    output:
        max_length: the max length of the sequences
    '''
    max_length = 0
    for i, seq in enumerate(sequences):
        length = len(seq)
        if max_length < length:
            max_length = length
    return max_length

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"

# Separate the sentences and the labels
train_x = list(X_train.clean_name)
train_y = np.array(X_train.purpose)
test_x = list(X_test.clean_name)
test_y = np.array(X_test.purpose)

# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

print("Example of sentence: ", train_x[4])

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
max_len = max_length(training_sequences)

print('Into a sequence of int:', training_sequences[4])

# Pad the sequence to have the same size
training_padded = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
print('Into a padded sequence:', training_padded[4])

Example of sentence:  kanthi akka pastry shop
Into a sequence of int: [2841, 2056, 463, 12]
Into a padded sequence: [2841 2056  463   12    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]


In [12]:
# See the first 10 words in the vocabulary
word_index = tokenizer.word_index
for i, word in enumerate(word_index):
    print(word, word_index.get(word))
    if i==9:
        break
vocab_size = len(word_index)+1
print(vocab_size)

<UNK> 1
ltd 2
pvt 3
lanka 4
the 5
sri 6
colombo 7
of 8
and 9
bank 10
12898


## LSTM Model

In [13]:
from tensorflow.keras import regularizers
from tensorflow.keras.constraints import MaxNorm

def define_model(input_dim = None, output_dim=300, max_length = None ):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(input_dim=input_dim, 
                                  mask_zero= True,
                                  output_dim=output_dim, 
                                  input_length=max_length, 
                                  input_shape=(max_length, )),
        
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=False)),
#         tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=8, activation='softmax')
    ])
    
    model.compile( loss = 'sparse_categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
#     model.summary()
    return model

In [14]:
model_0 = define_model( input_dim=1000, max_length=100)
model_0.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 300)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 128)          186880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 8)                 1032      
Total params: 586,728
Trainable params: 586,728
Non-trainable params: 0
_________________________________________________________________


In [15]:
class myCallback(tf.keras.callbacks.Callback):
    # Overide the method on_epoch_end() for our benefit
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.93):
            print("\nReached 93% accuracy so cancelling training!")
            self.model.stop_training=True


callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=0, 
                                             patience=20, verbose=2, 
                                             mode='auto', restore_best_weights=True)
# callbacks = myCallback()

## Train and Test the Model

In [16]:
# Parameter Initialization
trunc_type='post'
padding_type='post'
oov_tok = "<UNK>"


# # Separate the sentences and the labels
# train_x = list(corpus[corpus.split=='train'].sentence)
# train_y = np.array(corpus[corpus.split=='train'].label)
# test_x = list(corpus[corpus.split=='test'].sentence)
# test_y = np.array(corpus[corpus.split=='test'].label)


# encode data using
# Cleaning and Tokenization
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(train_x)

# Turn the text into sequence
training_sequences = tokenizer.texts_to_sequences(train_x)
test_sequences = tokenizer.texts_to_sequences(test_x)

max_len = max_length(training_sequences)

# Pad the sequence to have the same size
Xtrain = pad_sequences(training_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
Xtest = pad_sequences(test_sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)

word_index = tokenizer.word_index
vocab_size = len(word_index)+1

# Define the input shape
model = define_model(input_dim=vocab_size, max_length=max_len)

# Train the model
model.fit(Xtrain, train_y, batch_size=50, epochs=60, verbose=1, 
          callbacks=[callbacks], validation_data=(Xtest, test_y))

# evaluate the model
loss, acc = model.evaluate(Xtest, test_y, verbose=0)
print('Test Accuracy: {}'.format(acc*100))


Epoch 1/60
385/385 [==============================] - 44s 114ms/step - loss: 1.0497 - accuracy: 0.6328 - val_loss: 0.7193 - val_accuracy: 0.7508
Epoch 2/60
385/385 [==============================] - 38s 98ms/step - loss: 0.4450 - accuracy: 0.8647 - val_loss: 0.7235 - val_accuracy: 0.7610
Epoch 3/60
385/385 [==============================] - 37s 97ms/step - loss: 0.2383 - accuracy: 0.9294 - val_loss: 0.8002 - val_accuracy: 0.7562
Epoch 4/60
385/385 [==============================] - 38s 100ms/step - loss: 0.1748 - accuracy: 0.9469 - val_loss: 0.8654 - val_accuracy: 0.7400
Epoch 5/60
385/385 [==============================] - 43s 111ms/step - loss: 0.1343 - accuracy: 0.9579 - val_loss: 0.9131 - val_accuracy: 0.7535
Epoch 6/60
385/385 [==============================] - 42s 110ms/step - loss: 0.1089 - accuracy: 0.9659 - val_loss: 0.9656 - val_accuracy: 0.7442
Epoch 7/60
385/385 [==============================] - 40s 104ms/step - loss: 0.0946 - accuracy: 0.9671 - val_loss: 1.0550 - val_accu